In [12]:
OPENROUTER_API_KEY = "sk-or-v1- #Chave da API (não compartilhe)

MODEL_NAME = "mistralai/mistral-nemo:free" #Modelo de IA do OpenRouter

# Limites gerais
MAX_OUTPUT_TOKENS = 500       # tokens de saída (resposta)
MAX_CONTEXT_CHARS = 10000     # limite simples por caracteres para o contexto enviado

import unicodedata

github = '' #Link do GitHub quando houver um
contexto = ''

def _remove_accents(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))

def perguntar_openrouter(pergunta: str,
                         contexto: str,
                         model: str = MODEL_NAME,
                         max_tokens: int = MAX_OUTPUT_TOKENS,
                         referer: str = github,
                         app_title: str = "Assistente de IA") -> str:
    """Envia a pergunta com contexto ao endpoint do OpenRouter."""
    if not OPENROUTER_API_KEY or not OPENROUTER_API_KEY.startswith("sk-"):
        return "⚠️ Configure OPENROUTER_API_KEY corretamente."

    system_msg = (
        "Você é uma IA para teste"
    )

    user_prompt = f"""
[CONTEXTO]
{contexto}

1. Responda a pergunta com base no contexto acima.
2. O contexto é o histórico de mensagens entre você, marcado como 'Modelo', e o úsuario, marcado como 'Usuário'

[PERGUNTA]
{pergunta}
""".strip()

    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": referer,
        "X-Title": app_title,
    }

    body = {
        "model": model,
        "messages": [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_prompt},
        ],
        "max_tokens": max_tokens,
    }

    try:
        resp = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=body,          # usa json=body (requests serializa pra você)
            timeout=90,
        )
        resp.raise_for_status()
        data = resp.json()
        return data["choices"][0]["message"]["content"].strip()
    except requests.HTTPError as e:
        try:
            err = resp.json()
        except Exception:
            err = {"error": str(e)}
        return f"❌ Erro HTTP: {e}\nDetalhes: {err}"
    except Exception as e:
        return f"❌ Erro: {e}"


pergunta = input("Pergunte ou digite '1' para sair: ")

resposta = perguntar_openrouter(pergunta,contexto,MODEL_NAME,MAX_OUTPUT_TOKENS,github,'Teste')
print(f'Resposta: {resposta}')

contexto = contexto + f"Usuário: {pergunta}\nModelo: {resposta}\n"

Pergunte ou digite '1' para sair: 1 + 45
Resposta: O resultado de 1 + 45 é 46.
